In [44]:
import tensorflow as tf
import numpy as np
import cv2
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator

layers = tf.keras.layers
weight_decay = tf.keras.regularizers.l2(10e-4)
#weight_decay = None
useBais = False

In [45]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [46]:
# create a new generator
imagegen = ImageDataGenerator(rescale=1./255)
# load train data
train = imagegen.flow_from_directory("imagenette2/train/", class_mode="sparse", shuffle=False, batch_size=24, target_size=(224, 224))
# load val data
val = imagegen.flow_from_directory("imagenette2/val/", class_mode="sparse", shuffle=False, batch_size=24, target_size=(224, 224))

Found 9469 images belonging to 10 classes.
Found 3925 images belonging to 10 classes.


In [47]:
'''
DenseNet-121

- Growth rate (k): 32
- Compression rate: 0.5
- Blocks: [6,12,24,16]

'''

class DenseNet(tf.keras.Model):
    def __init__(self):
        super(DenseNet,self).__init__()
            
        
    def dense_block(self,x,block):
        
        for i in range(block):
            x = self.composite(x,32)        
        return x
        
    def composite(self,x,growth_rate):
        
        x_0 = x
        
        x = layers.BatchNormalization(epsilon=1.001e-5)(x)
        x = layers.Activation("relu")(x)
        x = layers.Conv2D(4*growth_rate,1,use_bias=useBais, kernel_regularizer= weight_decay,kernel_initializer="he_normal")(x)
        x = layers.BatchNormalization(epsilon=1.001e-5)(x)
        x = layers.Activation("relu")(x)
        x = layers.Conv2D(growth_rate,3,padding="same",use_bias=useBais, kernel_regularizer= weight_decay,kernel_initializer="he_normal")(x)
        
        x = layers.Concatenate()([x_0,x])
        
        return x
    
        
    def transition(self,x,compression):
        
        x = layers.BatchNormalization(epsilon=1.001e-5)(x)
        x = layers.Activation("relu")(x)
        x = layers.Conv2D(int(x.shape[-1] * compression),1,use_bias=useBais, kernel_regularizer= weight_decay,kernel_initializer="he_normal")(x)
        x = layers.AveragePooling2D(2,strides=2)(x)
        
        return x
        
    def call(self,x):
        
        x = layers.ZeroPadding2D(padding=((3, 3), (3, 3)))(x)
        x = layers.Conv2D(64, 7, strides=2, use_bias=useBais, kernel_regularizer= weight_decay,kernel_initializer="he_normal")(x)# = 2*k = 2 * 32 = 64.
        x = layers.BatchNormalization(axis=-1, epsilon=1.001e-5)(x)
        x = layers.Activation('relu', name='conv1/relu')(x)
        x = layers.ZeroPadding2D(padding=((1, 1), (1, 1)))(x)
        x = layers.MaxPooling2D(3, strides=2, name='pool1')(x)
        
        
        x = self.dense_block(x,6)
        x = self.transition(x,0.5)
        x = self.dense_block(x,12)
        x = self.transition(x,0.5)
        x = self.dense_block(x,24)
        x = self.transition(x,0.5)
        x = self.dense_block(x,16)
        
        x = layers.BatchNormalization(axis=-1, epsilon=1.001e-5)(x)
        x = layers.Activation('relu')(x)
        
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dense(10,activation="sigmoid")(x)
        
        return x

In [50]:
'''
FUNCTIONAL API IMPLEMENTATION

DenseNet-121

- Growth rate (k): 32
- Compression rate: 0.5
- Blocks: [6,12,24,16]

'''
def dense_block(x,block):

    for i in range(block):
        x = composite(x,32)        
    return x
        
def composite(x,growth_rate):

    x_0 = x

    x = layers.BatchNormalization(epsilon=1.001e-5)(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(4*growth_rate,1,use_bias=useBais, kernel_regularizer= weight_decay)(x)
    x = layers.Dropout(0.2)(x)
    
    x = layers.BatchNormalization(epsilon=1.001e-5)(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(growth_rate,3,padding="same",use_bias=useBais, kernel_regularizer= weight_decay)(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Concatenate()([x_0,x])

    return x
    

def transition(x,compression):

    x = layers.BatchNormalization(epsilon=1.001e-5)(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(int(x.shape[-1] * compression),1,use_bias=useBais, kernel_regularizer= weight_decay)(x)
    x = layers.Dropout(0.2)(x)
    x = layers.AveragePooling2D(2,strides=2)(x)

    return x

def DenseNetS():
        
    inputs = tf.keras.layers.Input(shape=(224,244,3))
    x = layers.ZeroPadding2D(padding=((3, 3), (3, 3)))(inputs)
    x = layers.Conv2D(64, 7, strides=2, use_bias=useBais, kernel_regularizer= weight_decay)(x)# = 2*k = 2 * 32 = 64.
    x = layers.BatchNormalization(axis=-1, epsilon=1.001e-5)(x)
    x = layers.Activation('relu')(x)
    x = layers.ZeroPadding2D(padding=((1, 1), (1, 1)))(x)
    x = layers.MaxPooling2D(3, strides=2,)(x)


    x = dense_block(x,6)
    x = transition(x,0.5)
    x = dense_block(x,12)
    x = transition(x,0.5)
    x = dense_block(x,24)
    x = transition(x,0.5)
    x = dense_block(x,16)

    x = layers.BatchNormalization(axis=-1, epsilon=1.001e-5)(x)
    x = layers.Activation('relu')(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(10,activation="softmax")(x)

    return tf.keras.Model(inputs,x)

In [ ]:
model = DenseNetS()
model.compile(loss="sparse_categorical_crossentropy",optimizer=tf.keras.optimizers.SGD(lr=0.0001,nesterov=True,momentum=0.9),metrics="acc")
model.fit(train,validation_data=val,epochs=100)

Epoch 1/100
395/395 [==============================] - 97s 245ms/step - loss: 17.8597 - acc: 0.0936 - val_loss: 17.8575 - val_acc: 0.1297
Epoch 2/100
395/395 [==============================] - 97s 245ms/step - loss: 17.8318 - acc: 0.0881 - val_loss: 17.8020 - val_acc: 0.0813
Epoch 3/100
395/395 [==============================] - 98s 247ms/step - loss: 17.8053 - acc: 0.0835 - val_loss: 17.7967 - val_acc: 0.0828
Epoch 4/100
395/395 [==============================] - 98s 248ms/step - loss: 17.7805 - acc: 0.0787 - val_loss: 17.7597 - val_acc: 0.0884
Epoch 5/100
395/395 [==============================] - 98s 249ms/step - loss: 17.7536 - acc: 0.0978 - val_loss: 17.7318 - val_acc: 0.0713
Epoch 6/100
395/395 [==============================] - 98s 248ms/step - loss: 17.7239 - acc: 0.1000 - val_loss: 17.7161 - val_acc: 0.1042
Epoch 7/100
395/395 [==============================] - 99s 249ms/step - loss: 17.7077 - acc: 0.0828 - val_loss: 17.6835 - val_acc: 0.0708
Epoch 8/100
395/395 [=============